### 2.1 Preliminary matters
#### 2.1.1 Install Gurobi
#### 2.1.2 Load Package JuMP and Gurobi

In [1]:
using JuMP
using Gurobi

#### 2.1.3 Set working directory to folder that contains data files

In [2]:
cd("../HW4_data")

In [3]:
pwd()

"/Users/lyynikki/Desktop/course/15.071C/homework/HW4_data"

#### 2.1.4 Read in train, validation, and test files

In [4]:
train = readcsv("Train.csv");
validation = readcsv("Validation.csv");
test = readcsv("Test.csv");

#### 2.1.5 Separate X and Y variables from each set

In [5]:
x_train = train[:,(1:11)];
x_validation = validation[:,(1:11)];
x_test = test[:,(1:11)];

y_train = train[:,12];
y_validation = validation[:,12];
y_test = test[:,12];

### 2.2 Data Cleaning
#### 2.2.1 Normalize each column in the training data

In [6]:
#2.2.1.a Compute the mean of y train, and each column of x train
x_train_mean = mean(x_train,1);
y_train_mean = mean(y_train);
#2.2.1.b Subtract these mean values from y train and each column
x_train = x_train .- x_train_mean;
y_train = y_train - y_train_mean;
#2.2.1.c Compute the norm of each column of x train
#2.2.1.d Divide each column of x train by its norm
x_train_norm = ones(11);
for i in 1:11
    x_train_norm[i] = vecnorm(x_train[:,i],2)
    x_train[:,i] = x_train[:,i]/x_train_norm[i]
end;

#### 2.2.2 Normalize the validation and test set

In [7]:
# normalize validation
x_validation = x_validation .- x_train_mean;
y_validation = y_validation - y_train_mean;
for i in 1:11
    x_validation[:,i] = x_validation[:,i]./x_train_norm[i]
end;

#normalize test
x_test = x_test .- x_train_mean;
y_test = y_test - y_train_mean;
for i in 1:11
    x_test[:,i] = x_test[:,i]./x_train_norm[i]
end;

### 2.3 Starting the model
#### 2.3.1 Define the constant D to be the number of independent variables

In [8]:
D = size(x_train)[2];
a = size(x_train)[1];

#### 2.3.2 Create a model objecting using Model() and the solver you intend to use

In [9]:
using JuMP, Gurobi
model = Model(solver=GurobiSolver());

#### 2.3.3 Define your Beta variables using @variable, to have length D

In [10]:
@variable(model, beta[1:D])

11-element Array{JuMP.Variable,1}:
 beta[1] 
 beta[2] 
 beta[3] 
 beta[4] 
 beta[5] 
 beta[6] 
 beta[7] 
 beta[8] 
 beta[9] 
 beta[10]
 beta[11]

#### 2.3.4 Define binary indicator variables z

In [11]:
@variable(model, z[1:D],Bin)

11-element Array{JuMP.Variable,1}:
 z[1] 
 z[2] 
 z[3] 
 z[4] 
 z[5] 
 z[6] 
 z[7] 
 z[8] 
 z[9] 
 z[10]
 z[11]

### 2.4 Adding constraints
#### 2.4.1 Set constraints
###### 2.4.1.1 Purpose of the z variable:
Z variables are binay variables. If z(i) = 0, it forces the corresponding beta(i) to be 0, which means the corresponding X(i) variable will not appear in the regression equation. Therefore, z variables enable us to constraint the existence of its corresponding X variables. Therefore, it restrict how many X variables will appear in the model and which X variables will appear in the model.
###### 2.4.1.2 Big M
Big M represents a very large integer. We use M to set a range for beta, which is: -z[i]*M <= beta[i] <= z[i]*M for all i. z[i] is binary variable. When z(i) is zero, beta(i) have to be zero, and X(i) won't appear in the model. When z(i)=1, since M is very large, there is actually no extra constraint for beta(i). The big M should be so large that no matter what beta[i]'s real value is, it can fall into the range of minus M to M. 
###### 2.4.1.3 Fix a value for your “big M” and formulate the constraints
I would choose M to be 100000

In [12]:
M = 100000;
for i in 1:D
    @constraint(model, -z[i]*M <= beta[i] )
    @constraint(model, beta[i] <= z[i]*M)
end

#### 2.4.2 Write a constraint that enforces sparsity in your model. Set the maximumnumber of variables used to 5.

In [13]:
@constraint(model,sum(z) <=5)

z[1] + z[2] + z[3] + z[4] + z[5] + z[6] + z[7] + z[8] + z[9] + z[10] + z[11] ≤ 5

#### 2.4.3 Write constraints to prevent multicollinearity in the model

In [14]:
@constraint(model,z[1]+z[2] <=1)
@constraint(model,z[5]+z[6] <=1)

z[5] + z[6] ≤ 1

#### 2.4.4 Write a constraint to exclude the solution where the first five variables are all used.

In [15]:
@constraint(model,z[1]+z[2]+z[3]+z[4]+z[5] <=4)

z[1] + z[2] + z[3] + z[4] + z[5] ≤ 4

### 2.5 Defining the objective
#### 2.5.1 Write out in the comments an expression for the sum of squared error we want to minimize, using y train, x train, and the variable Beta.
What we want to minimize is:

minimize over beta: sum(y_train[i] - x_train[i,:]'*beta)^2    for i in 1:73
#### 2.5.2 Formulate the expression in Julia

In [16]:
@variable(model, e[1:a])
for j = 1:a
    @constraint(model, y_train[j] - sum(x_train[j,i]*beta[i] for i = 1:D) == e[j])
end  

In [17]:
@objective(model, Min, sum(e[j]^2 for j = 1:a))

:Min

### 2.6 Running the model
#### 2.6.1 Solve the model

In [18]:
solve(model);

Optimize a model with 99 rows, 95 columns and 940 nonzeros
Model has 73 quadratic objective terms
Coefficient statistics:
  Matrix range    [4e-04, 1e+05]
  Objective range [0e+00, 0e+00]
  Bounds range    [1e+00, 1e+00]
  RHS range       [2e-02, 5e+00]
Found heuristic solution: objective 110.545
Presolve removed 1 rows and 0 columns
Presolve time: 0.01s
Presolved: 98 rows, 95 columns, 935 nonzeros
Presolved model has 73 quadratic objective terms
Variable types: 84 continuous, 11 integer (11 binary)

Root relaxation: objective 1.911635e+00, 128 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    1.91163    0    6  110.54518    1.91163  98.3%     -    0s
     0     0    1.91253    0    5  110.54518    1.91253  98.3%     -    0s
H    0     0                       2.4296317    1.91253  21.3%     -    0s
     0     2    1.91253    0    3    2.42963   

#### 2.6.2 Save the value of Beta

In [19]:
est_beta = getvalue(beta)

11-element Array{Float64,1}:
 -2.38034 
  0.0     
  0.0     
 -3.77384 
  1.97906 
  0.0     
  0.0     
  0.0     
  0.0     
 -4.96516 
  0.900082

#### 2.6.3 Create predictions for y test

In [20]:
y_test_pred = x_test * est_beta;

#### 2.6.4 Calculate the sum of squared error of your model on the test set

In [21]:
sse = sum((y_test_pred .- y_test).^2)

1.7694917963608077

sum of squared error is 1.769
#### 2.6.5 Compute the sum of squared error of the baseline model on the test set
The baseline model here is the naive prediction for the mean in the training set

In [22]:
y_mean = mean(y_train)

5.840077280220001e-16

The naive prediction would be for all observations in the testing set, predict y = 5.840*e-16, which is close to 0.

In [23]:
sse_baseline = sum((y_test .- y_mean).^2)

35.64438829836743

sum of squared error is 35.644
#### 2.6.6 Compute the R2 of your model

In [24]:
# for training set:
y_train_pred = x_train * est_beta;
r2_train = 1 - (sum((y_train .- y_train_pred).^2))/(sum((y_train .- y_mean).^2))

0.9780213699538429

In [25]:
# for training set:
y_vali_pred = x_validation * est_beta;
r2_vali = 1 - (sum((y_validation .- y_vali_pred).^2))/(sum((y_validation .- y_mean).^2))

0.9635134427813272

In [26]:
# for training set:
r2_test = 1 - (sum((y_test_pred .- y_test).^2))/(sum((y_test .- y_mean).^2))

0.9503570721553987

so, the R square for traing, validation and testing are :
0.978, 0.964 and 0.950.

### 2.7 EXTRA CREDIT
#### 2.7.3 Show how you might select the sparsity
I would run a loop for sparcity level from 1 to 11

In [27]:
## I am going to construct an array, containing 11 models, each model corresponding to 
## a different level of sparsity
c = Model(solver=GurobiSolver());
model = [c,c,c,c,c,c,c,c,c,c,c];
## for each model, there is a set of est_beta value, and I will store betas in est_beta matrix
est_beta = zeros(11,11);
## I want to record sse and R squre for validation set 
sse_vali = zeros(11);
R2_vali = zeros(11);

#### Run a loop over various sparsity levels

In [28]:
## k is the sparsity level, which is how many variables the model includes
for k = 1:11
    mm = model[k]
    @variable(mm, beta[1:D]);
    @variable(mm, z[1:D],Bin);
    for i in 1:D
        @constraint(mm, -z[i]*M <= beta[i])
        @constraint(mm, beta[i] <= z[i]*M)
    end;
    @constraint(mm,sparsity,sum(z) <= k);
    @constraint(mm,z[1]+z[2] <=1);
    @constraint(mm,z[5]+z[6] <=1);
    @constraint(mm,z[1]+z[2]+z[3]+z[4]+z[5] <=4);
    @variable(mm, e[1:a]);
    for j = 1:a
        @constraint(mm, y_train[j] - sum(x_train[j,i]*beta[i] for i = 1:D) == e[j])
    end ;
    @objective(mm, Min, sum(e[j]^2 for j = 1:a));
    solve(mm);
    est_beta[k,:] = getvalue(beta);
    sse_vali[k] =  sum((y_validation .- x_validation * est_beta[k,:]).^2);
    R2_vali[k] = 1 - (sum((y_validation .- x_validation * est_beta[k,:]).^2))/(sum((y_validation .- y_mean).^2))
end   

Optimize a model with 99 rows, 95 columns and 940 nonzeros
Model has 73 quadratic objective terms
Coefficient statistics:
  Matrix range    [4e-04, 1e+05]
  Objective range [0e+00, 0e+00]
  Bounds range    [1e+00, 1e+00]
  RHS range       [2e-02, 4e+00]
Found heuristic solution: objective 110.545
Presolve removed 3 rows and 0 columns
Presolve time: 0.00s
Presolved: 96 rows, 95 columns, 931 nonzeros
Presolved model has 73 quadratic objective terms
Variable types: 84 continuous, 11 integer (11 binary)

Root relaxation: objective 1.911635e+00, 128 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    1.91163    0    6  110.54518    1.91163  98.3%     -    0s
*    0     0               0      95.7195199   95.71952  0.00%     -    0s

Explored 0 nodes (141 simplex iterations) in 0.01 seconds
Thread count was 4 (of 4 available processors)

Optimal solutio

Optimize a model with 198 rows, 190 columns and 1880 nonzeros
Model has 73 quadratic objective terms
Coefficient statistics:
  Matrix range    [4e-04, 1e+05]
  Objective range [0e+00, 0e+00]
  Bounds range    [1e+00, 1e+00]
  RHS range       [2e-02, 4e+00]
Found heuristic solution: objective 110.545
Presolve removed 100 rows and 95 columns
Presolve time: 0.00s
Presolved: 98 rows, 95 columns, 935 nonzeros
Presolved model has 73 quadratic objective terms

Loaded MIP start with objective 5.23576

Variable types: 84 continuous, 11 integer (11 binary)

Root relaxation: objective 1.911635e+00, 128 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    1.91163    0    6    5.23576    1.91163  63.5%     -    0s
     0     0    1.91253    0    5    5.23576    1.91253  63.5%     -    0s
     0     2    1.91253    0    5    5.23576    1.91253  63.5%     -    0s

#### Select the sparsity that minimizes out-of-sample error on the validation set
Minimize validation sum of squred error or maximize validation R squre:

In [29]:
[sse_vali,R2_vali]

2-element Array{Array{Float64,1},1}:
 [33.6586,2.85705,1.61944,1.54522,1.59348,2.30908,2.4343,2.4343,2.18744,2.18744,2.16118]             
 [0.229309,0.934581,0.962919,0.964619,0.963513,0.947128,0.944261,0.944261,0.949914,0.949914,0.950515]

I use validation R square and validation sum of squred error as metric, the best sparsity level is 4.

When choosing sparsity level = 4

In [30]:
sse_test_4 = sum((y_test .- x_test * est_beta[4,:]).^2)
R2_test_4 = 1 - (sum((y_test .- x_test * est_beta[4,:]).^2))/(sum((y_test .- y_mean).^2))
sse_test_4,R2_test_4

(1.7489137873655933,0.9509343862847074)

The test set sum squred error = 1.749, and test R square = 0.951